# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder)

In [4]:
from dvats.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader

## Config parameters

Put here everything that could be needed if this notebook

In [5]:
import utils.config as cfg
config, job_type = cfg.get_artifact_config_embeddings(True)

Current: /home/macu/work/nbs_pipeline
yml: ./config/03-embeddings.yaml
yml: ./config/03-embeddings.yaml
Getting content./config/03-embeddings.yaml
... About to replace includes with content
Load content./config/03-embeddings.yaml
enc_artifact: mi-santamaria/deepvats/mvp:latest


## Run

In [6]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="03_embeddings"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
print("runname: "+runname)

runname: 03_embeddings


In [7]:
run = wandb.init(
    entity      = config.wandb_entity,
    project     = config.wandb_project if config.use_wandb else 'work-nbs', 
    group       = config.wandb_group,
    job_type    = job_type,
    mode        = 'online' if config.use_wandb else 'disabled',
    anonymous   = 'never' if config.use_wandb else 'must',
    config      = config,
    resume      = 'allow',
    name        = runname
)

wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


'stream.Stream' object attribute 'write' is read-only


In [8]:
# Botch to use artifacts offline
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

Restore the encoder model and its associated configuration

In [9]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')

In [10]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()

wandb:   1 of 1 files downloaded.  


Restore the dataset artifact used for training the encoder. Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that 
it matches with the dataset that we want to reduce.

In [11]:
enc_run = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_run.config['train_artifact'], type='dataset')
enc_artifact_train.name

'ETTh1:v0'

Now we specify the dataset artifact that we want to get the embeddings from. If no 
artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [12]:
input_ar_name = ifnone(config.input_ar, 
                       f'{enc_artifact_train.entity}/{enc_artifact_train.project}/{enc_artifact_train.name}')
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

'ETTh1:v0'

In [13]:
df = input_ar.to_df()
df

wandb:   1 of 1 files downloaded.  


,MULL,LUFL,LULL,OT
date,,,,
2016-07-01 00:00:00,0.462,4.203,1.340,30.531000
2016-07-01 01:00:00,0.426,4.142,1.371,27.787001
2016-07-01 02:00:00,0.355,3.777,1.218,27.787001
2016-07-01 03:00:00,0.391,3.807,1.279,25.044001
2016-07-01 04:00:00,0.462,3.868,1.279,21.948000
...,...,...,...,...
2018-06-26 15:00:00,2.132,3.472,1.523,10.904000
2018-06-26 16:00:00,2.274,3.533,1.675,11.044000
2018-06-26 17:00:00,2.097,3.716,1.523,10.271000


In [14]:
df.shape

(17420, 4)

In [15]:
enc_input, _ = SlidingWindow(window_len=enc_run.config['w'], 
                             stride=enc_run.config['stride'], 
                             get_y=[])(df)
enc_input.shape

(17391, 4, 30)

In [16]:
embs = get_enc_embs(enc_input, enc_learner, cpu=config.cpu, to_numpy=True)

In [17]:
if config.use_wandb: 
    run.log_artifact(ReferenceArtifact(embs, 'embeddings', metadata=dict(run.config)), 
                     aliases=f'run-{run.project}-{run.id}')

In [18]:
run.finish()